### 0: Libraries

In [9]:
import requests
import pandas as pd
import geopandas as gpd
import json
from datetime import datetime, timedelta
import os
import glob
import warnings
import psycopg2
import sqlalchemy as db
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from shapely.geometry import Point
import contextily as ctx
from geoalchemy2 import Geometry
from pathlib import Path
import ipywidgets as widgets
from ipywidgets import interact

import unittest
from unittest.mock import patch
# warnings
warnings.filterwarnings('ignore')

# Part1: Data Processing

## 1.1 Downloading 311 and Tree data from API
- Download 311 and Tree data using API and Python Code
- 311 Data set is pretty huge, so we choose to download it seperately into subfiles by year, and merge it at the end